The purpose of this code is to explore different configurations for a radar-like spectroscopy system. The battery will be simulated by a set of different impedances that are separated from each other. As shown in the diagram below

![title](battery.png)

Each part of the battery has a complex impedance (per unit length, just like a transmission line) and a length. Thus, the wave propagates as
\begin{equation}
A(z,t) = A_i\exp\left(-j\omega t - \gamma z\right)
\end{equation}

where $\gamma = \sqrt{ZY}$ is the [b]complex[\b] propagation constant, where $Y$ is the [b]shunt[\b] admitance and $Z$ is the [b]series[\b] impedance. The propgation constant can be split into a real and imaginary part $\gamma = \alpha + j\beta$. This leads to
\begin{equation}
A(z,t) = A_i\exp\left(-\alpha z\right)\exp\left(-j\left(\omega t - \beta z\right)\right)
\end{equation}
where $\alpha = \textrm{Re}\sqrt{ZY}$ and $\beta = \textrm{Im}\sqrt{ZY}$. So, the big questions for determining how the attenuation and phase change of the wave are what are $Z$ and $Y$. $Z = R + j\omega L$, the series impedance of the different battery components. $Y = G + j\omega C$, where $G$ is the shunt conductance and $C$ is the shunt capacitance. In general $G\approx 0$ for all components and $R\approx 0$ for all components (at least at low frequency), while $C$ is quite large for the electrodes, and quite small for the electrolyte.

At the interface between two battery components with impedances, $Z_1$ and $Z_2$, there will be both reflection and transmission. The amplitude reflection coefficient for a wave travelling from $Z_1$ to $Z_24 is given by
\begin{equation}
\rho = \frac{Z_2 - Z_1}{Z_2 + Z_1}
\end{equation}
and the transmission is given by
\begin{equation}
\tau = \frac{2Z_2}{Z_1 + Z_2}
\end{equation}


So, let's set up some starting structure. We need some packages. And we will define the battery as a class with a bunch of methods and properties. To create a battery we first decide how many elements we want. Then, for each element, we define the shunt admitance and series impedance and length. Each element is defined by a name that we can use to reference it later if we want to change aspects of its properties (it is a dictionary that is given a 5 element tuple).

The methods of the battery class also allow you to calculate the propagation constant for an element and to calculate the reflection/transmission at the boundary between elements. This same method has optional arguments to calculate reflection/transmission at the entrance and exit of the battery

In [145]:
import numpy as np #library for numerical calculations
from scipy.constants import c, epsilon_0 #physical constants that we will probably need
pi = np.pi

class battery:
    #initialization is only the number of elements, but not their properties
    def __init__(self, num_elements):
        self.el_dict = dict() #holds properties of all elements
        self.el_vals_set = False #used to test if all elements have properties (not checked everywhere)
        self.el_count = 0 #used to keep track of which element needs to be set
        self.num_elements = num_elements #total number of elements in battery
        self.A_lr = np.zeros((num_elements + 2,), complex) #complex amplitude traveling left to right
        self.A_rl = np.zeros((num_elements + 2,), complex) #complex amplitude travelling right to ledt
        
        return
    #method to set each element's properties. If el_number is not specified, it specifies new element
    #otherwise el_number lets you replace the ith element with new properties
    #elements are specified by name and consist of a series resistance, a series inductance and parallel conductance
    #a parallel capacitance and a physical length (so a lossy transmission line of length l)
    def setElement(self, name, R, G, L, C, l, el_number = -1):
        self.el_dict[name] = (R, G, L, C, l)
        if el_number == -1: #setting the next element
            self.el_count += 1
        if self.el_count == self.num_elements:
            self.el_vals_set = True
        return
    #calculate the propagation constants of an element (frequency dependent)
    def calcPropagationConstants(self, element, freq):
        Z = (self.el_dict[element])[0] + 1.0j*2*np.pi*freq*(self.el_dict[element])[2] #line impedace
        Y = (self.el_dict[element])[1] + 1.0j*2*np.pi*freq*(self.el_dict[element])[3] #admitance
        gamma = np.sqrt(Z*Y)
        alpha = gamma.real #absorption
        beta = gamma.imag #propagation
        c_el = 2*np.pi*freq/beta
        return alpha, beta, c_el #speed of light in element
    #calculate the reflection and transmission amplitude coefficients between two elements
    #set element = None for entrance and exit locations
    def calcReflectionAndTransmission(self, el_1, el_2, freq,  Z0 = 50):
        #print (el_1, el_2)
        if el_1 == None:
            Z1 = Z0
        else:
            Z1 = self.el_dict[el_1][0] + 1.0j*2*np.pi*freq*self.el_dict[el_1][2]
        if el_2 == None:
            Z2 = Z0
        else:
            Z2 = self.el_dict[el_2][0] + 1.0j*2*np.pi*freq*self.el_dict[el_2][2]

        rho_lr = (Z2 - Z1)/(Z1 + Z2) # reflection left to right
        tau_lr = 2*Z2/(Z1 + Z2) #transmission left to right
        rho_rl = (Z1 - Z2)/(Z1 + Z2) #reflection right to left (note difference)
        tau_rl = 2*Z1/(Z1 + Z2) #transmission right to left (note difference)
        #print(rho_lr, tau_lr, rho_rl, tau_rl)
        return rho_lr, rho_rl, tau_lr, tau_rl
    #calculate the change in amplitude (including phase change) due to traversing an element (requires a frequency)
    #if exiting or entering the battery set element to None
    def calcElementPropagation(self, j, element, freq, lr):
        if element == None:
            return
        else:
            alpha, beta, c_el = self.calcPropagationConstants(element, freq)
            dt = self.el_dict[element][4]/c_el
            if lr:
                self.A_lr[j] = self.A_lr[j]*np.exp(-1.0j*(2*pi*freq*dt - beta*self.el_dict[element][4]))*np.exp(-alpha*self.el_dict[element][4])
            else:
                self.A_rl[j] = self.A_rl[j]*np.exp(-1.0j*(2*pi*freq*dt - beta*self.el_dict[element][4]))*np.exp(-alpha*self.el_dict[element][4])
        return 
    #calculates the radiation balance between two elements.
    #j indicates the element on the left of the boundary
    def calcBoundaryBalance(self, j, freq, Z0 = 50):
        ks = list(self.el_dict.keys())
        #print (j, self.num_elements, len(ks))
        if j == 0:
            key_left = None
            key_right = ks[j]
        elif j == (self.num_elements):
            key_right = None
            key_left = ks[j-1]
        else:
            key_right = ks[j]
            key_left = ks[j-1]
        #print(key_left, key_right)
        rho_lr, rho_rl, tau_lr, tau_rl = self.calcReflectionAndTransmission(key_left, key_right, freq)
            
        #print(self.A_lr[j], self.A_rl[j+1])
        #add transmission from left and reflection from right travelling radiation
        self.A_lr[j+1] = tau_lr*self.A_lr[j] + rho_rl*self.A_rl[j+1]
        #print(self.A_lr[j+2])
        self.calcElementPropagation(j+1, key_right, freq, True)
        #add transmission from right and reflection from left travelling radiation
        self.A_rl[j] = tau_rl*self.A_rl[j+1] + rho_lr*self.A_lr[j]
        #print(self.A_rl[j])
        self.calcElementPropagation(j, key_right, freq, False)
        
        return 
    #this function checks for convergence
    def calcSum(self):
        a1 = np.abs(self.A_lr).sum()
        a2 = np.abs(self.A_rl).sum()
        return a1, a2
    #this function calculates the steady state amplitudes within the battery and at the entrance and exit.
    #lr sets the propagation direction (so full S parameters can be calculated
    #lr = True --> only left to right calculated
    #lr = False --> only right to left calculated
    def calcSteadyState(self, freq, Z0, lr=True):
        err = 1e6
        if lr:
            self.A_lr[0] = 1 #need to reset the boundary conditions at the start of each iteration (constant input from left)
            #need to reset the boundary conditions at the start of each iteration (no input from right)
            self.A_rl[self.num_elements+1] = 0
        else:
            self.A_lr[0] = 0
            self.A_rl[self.num_elements+1] = 1
                
        while err > 1e-9:
            a1 = max(self.calcSum())
            for j in range(self.num_elements+1):
                self.calcBoundaryBalance(j, freq, Z0)
            if lr:
                #print(self.A_lr[self.num_elements+1])
                self.A_lr[0] = 1 #need to reset the boundary conditions at the start of each iteration (constant input from left)
                #need to reset the boundary conditions at the start of each iteration (no input from right)
                self.A_rl[self.num_elements+1] = 0
            else:
                self.A_lr[0] = 0
                self.A_rl[self.num_elements+1] = 1
                
            a2 = max(self.calcSum())
            err = abs(a1 - a2)
            #print(err)
            
#print(self.A_lr)
            #print(self.A_rl)
        
                
        
        
        


Now that we have defined our class, we should instantate it with a battery and some radiation


In [146]:
my_battery = battery(5)
#note that the numbers chosen in the following are not related to real batteries at this point. They are to test the code
my_battery.setElement("cathode", 1e-3, 1e-6, 1e-9, 1e-6, 250e-6) 
my_battery.setElement("cathode interface", 5e-3, 1e-9, 1e-8, 5e-6, 2.5e-6)
my_battery.setElement("electrolyte", 1.0, 1e-9, 10e-9, 10e-6, 3e-3)
my_battery.setElement("anode interface", 10e-3, 1e-9, 1e-8, 5e-6, 25e-6)
my_battery.setElement("anode", 1e-3, 1e-6, 1e-9, 1e-6, 250e-6)


freq_rf = 100e9


The class, when _calcSteadyState_ is run, will hold the radiation distribution, but for now we are interested in the _S_ parameters. To do that, we need to send radiation in from both ends and examine the complex amplitude of the reflected and transmitted waved. These are the first and last elements of the radation distribution.

However, the class only calculates the steady state in a single direction, so we need to calculate twice with the radiation going left to right, and then right to left. This is done by toggling _lr_.

In [148]:
my_battery.calcSteadyState(freq_rf, 50, lr = True)
#print(my_battery.A_lr)
#print(my_battery.A_rl)
r_in = my_battery.A_lr[0]
r_ref = my_battery.A_rl[0]
r_tr = my_battery.A_lr[my_battery.num_elements+1]

my_battery.calcSteadyState(freq_rf, 50, lr = False)
#print(my_battery.A_lr)
#print(my_battery.A_rl)
l_in = my_battery.A_rl[my_battery.num_elements+1]
l_ref = my_battery.A_lr[my_battery.num_elements+1]
l_tr = my_battery.A_rl[0]
print(r_in, r_ref, r_tr, l_in, l_ref, l_tr)

(1+0j) (0.8987755995130589+0.30176861444200037j) (0.10122057180347604-0.3013914417181981j) (1+0j) (0.8987792154663631+0.30176967265330323j) (0.10122041245264292-0.3013914511254007j)


Now, we have all the data required to calculate the _S_ parameters for a single frequency. If we want to calculate an _S_ parameter spectrum, we will need to calculate over a range of frequencies. It is also possible to simulate nonlinear performance by changing the element properties for each frequency.

In [150]:
def SParameterCalc(r_in, r_ref, r_tr, l_in, l_ref, l_tr):
    S11 = r_ref/r_in
    S12 = r_tr/r_in
    S21 = l_tr/l_in
    S22 = l_ref/l_in
    return S11, S12, S21, S22

S11, S12, S21, S22 = SParameterCalc(r_in, r_ref, r_tr, l_in, l_ref, l_tr)
print(S11, S12, S21, S22)

(0.8987755995130589+0.30176861444200037j) (0.10122057180347604-0.3013914417181981j) (0.10122041245264292-0.3013914511254007j) (0.8987792154663631+0.30176967265330323j)
